Please press the *Restart kernel* (fast forward sign) button until the user interface shows up at the bottom. 

In [1]:
%%html
<style>
div.input {
    display:none;
}
</style>

In [2]:
import math
import numpy as np
def knowledge_base(parameters, procedure):
    """
    Contains all cutoff values, compares the patient's values to the cut-off 
    values and then uses Boolean statements to indicate whether a patient has a
    certain pathology or not. 
    """
    # {key: [co_val, '>', [LB,UB]]} # LB/UB in fractions
    bnd = [1/1.1,1/0.9] #standard boundaries
    pathologies=[] #master dictionary, contains all cutoff values, is adjusted based on which procedure is followed
    pathologies.append({'Rotem_CT_Intem'  : [240,'>=',bnd], #Deficiencies in plasmatic coagulation factors
                      'Rotem_CT_Extem'  : [90,'>=',bnd],
                      'APTT'            : [50,'>=',bnd],	
                      'APTT_Heparinase' : [50,'>=',bnd],
                      'PT'              : [22,'>=',bnd],
                      'PT_Heparinase'   : [22,'>=',bnd],	
                      'Rotem_CT_Heptem' : [280,'>=',bnd]
                      })
    pathologies.append({'Fibrinogen'  : [1.5, '<=',bnd], #Lowered fibrinogen levels
                      ('Rotem_A10_Extem','Rotem_A10_Fibtem')  : [40, '<=',bnd,8, '<=',bnd]
                      })
    pathologies.append({('Rotem_CT_Intem','Rotem_CT_Heptem') : [240,'>=',bnd,240,'<=',bnd],  #Patient is heparinased
                      'CThep/CTin'      : [0.8, '<=',bnd]
                      })
    pathologies.append({'Thrombocytes' : [100, '<=',bnd], #Thrombocyte disorder
                      ('Rotem_A10_Heptem','Rotem_A10_Fibtem') : [30, '<=',bnd, 9,'>=',bnd],
                      'MEA_TRAP'    : [50, '<=',[1/1.2,1/0.8]],
                      'MEA_ADP'     : [40, '<=',[1/1.2,1/0.8]],
                      ('Rotem_A10_Extem','Rotem_A10_Fibtem')  : [40, '<=',bnd, 10, '>=',bnd],
                      })
    
    consult = {1 : 'Consider 1x5E platelet concentrate',
               2 : '1x5E platelet concentrate',
               3 : 'Consider fibrinogen',
               4 : '2-4 g fibrinogen',
               5 : '2-4 g fibrinogen or 10-15 ml/kg plasma',
               6 : '2 g fibrinogen',
               7 : '4 g fibrinogen',
               8 : '50 mg protamine',
               9 : '25 mg protamine',
               10: '0.3 mg/kg protamine',
               11: 'Consider plasma or Beriplex',
               12: '10-15 ml/kg plasma or 20-30 u/kg Beriplex' 
               }
    
    #Make adjustments to the master dictionary
    if procedure[0] == 'CTC/IORT':
        del pathologies[0]['Rotem_CT_Intem']
        if procedure[1] == 'intra':
            pathologies[0] = {}
            pathologies[1] = {'Fibrinogen'  : [0.8, '<=',bnd],
                              'Rotem_A10_Fibtem'     : [5, '<=',bnd]}
            pathologies[2] = {}    
            #del pathologies[3]['MEA_ADP']
            del pathologies[3][('Rotem_A10_Extem','Rotem_A10_Fibtem') ]
        elif procedure[1] == 'post' or procedure[1] == 'IC':
            del pathologies[3][('Rotem_A10_Heptem','Rotem_A10_Fibtem')]
        elif procedure[1] == 'IORT':
            pathologies[2] = {}
            del pathologies[3][('Rotem_A10_Heptem','Rotem_A10_Fibtem')]
            
            
    if procedure[0] == 'Excessive blood loss':
        pathologies[0]['APTT'][0] = 75
        pathologies[0]['APTT_Heparinase'][0] = 75
        pathologies[0]['PT'][0] = 33
        pathologies[0]['PT_Heparinase'][0] = 33
        pathologies[0]['Rotem_CT_Extem'][0] = 80
        pathologies[1] = {('Rotem_A10_Extem','Rotem_A10_Fibtem')  : [40, '<=', bnd,10, '>=',bnd]}
        del pathologies[3][('Rotem_A10_Heptem','Rotem_A10_Fibtem')]
        pathologies[3]['Thrombocytes'][0] = 50
    
    #Use inference engine
    state = ['Plasma defects', 'Low fibrinogen', 'Patient is heparinized','Thrombocyte disorder']
    curr_state = [{},{},{},{}]
    sev = [{},{},{},{}] #severity
    for i in range(len(pathologies)):
        for j in pathologies[i]:
            if i == 3:
                if True in curr_state[0].itervalues():
                    pathologies[3]['MEA_ADP'] = [35, '<=',[1/1.2,1/0.8]]
            tmp = compare_values(j,pathologies[i][j],parameters)
            curr_state[i][j] = tmp[0]
            sev[i][j] = tmp[1]
            
    #Consult
    consult_nr = []
    if procedure[0] == 'CTC/IORT':
        if procedure[1] == 'pre':
            if True in curr_state[3].itervalues():
                consult_nr.append(1)
            if True in curr_state[1].itervalues():
                consult_nr.append(3)
            if True in curr_state[0].itervalues():
                consult_nr.append(11)
        if procedure[1] == 'intra':
            if True in curr_state[3].itervalues():
                consult_nr.append(1)
            if True in curr_state[1].itervalues():
                consult_nr.append(4)            
        if procedure[1] == 'post' or procedure[1] == 'IC' or procedure[1] == 'IORT':
            if True in curr_state[3].itervalues():
                consult_nr.append(2)
            if True in curr_state[1].itervalues():
                consult_nr.append(5)
            if (procedure[1] == 'post' or procedure[1] == 'IC') and True in curr_state[2].itervalues():
                if procedure[1] == 'post':
                    consult_nr.append(8)
                if procedure[1] == 'IC':
                    consult_nr.append(9)
            if True in curr_state[0].itervalues():
                consult_nr.append(12)
    
    if procedure[0] == 'Excessive blood loss':
        if True in curr_state[3].itervalues():
            consult_nr.append(2)
        if True in curr_state[1].itervalues():
            if procedure[1] == 'HD stable':
                consult_nr.append(6)
            if procedure[1] == 'HD unstable':
                if parameters['Rotem_A10_Fibtem'] <= 7 or parameters['Fibrinogen'] <= 1:
                    consult_nr.append(7)
                else:
                    consult_nr.append(6)
        if True in curr_state[2].itervalues():
            consult_nr.append(10)            
        if True in curr_state[0].itervalues():
            consult_nr.append(12)
            
    print('Advice:')       
    for i in consult_nr:
        print(consult[i])
    print('\n')
    return curr_state, sev, pathologies 
    
def compare_values(key,value,parameters): #patient = {key: pat_val} cutoff = {key:[co_val, comp]}
    """
    Compares the patient's values with the cutoff values of a given
    parameter. Inference engine. 
    """
    # 0) none exceeded -> green
    # 1) LB exceeded, co/UB not -> requires attention -> orange
    # 2) LB & co exceeded, UB not -> pathologic state but within error ranges -> red
    # 3) LB, co & UB, definitely pathologic state -> dred
    
    
    runs = int(len(value)/3) #number of parameters that need to be evaluated, in the case of ands
    l = [] #stores whether a cutoff value has been exceeded or not, e.g. l=[[True,True,False]]
    for i in range(runs):
        if runs == 2:
            pat_val = parameters[key[i]]
        else:
            pat_val = parameters[key]
        co = value[3*i]
        comp = value[1+3*i]
        LB = value[2+3*i][0]
        UB = value[2+3*i][1]
        
        co_val = [LB*co,co,UB*co] #afronden?
        x = [] #stores whether the values in co_val have been exceeded or not, e.g. x=[True,True,False]
    
        for j in range(len(co_val)):        
            if math.isnan(pat_val):
                return ['N/A', 'N/A']
            if runs == 2 and (math.isnan(parameters[key[0]]) or math.isnan(parameters[key[1]])):
                return ['N/A', 'N/A']
            
            if comp == '>':
                if pat_val > co_val[j]:
                    x.append(True)
                else: 
                    x.append(False)
            if comp == '<':
                if pat_val < co_val[j]:
                    x.append(True)
                else: 
                    x.append(False)
            if comp == '<=':
                if pat_val <= co_val[j]:
                    x.append(True)
                else: 
                    x.append(False)
            if comp == '>=':
                if pat_val >= co_val[j]:
                    x.append(True)
                else: 
                    x.append(False)
     
        l.append(x) 
    p = []
    
    for k in range(len(l)):
        if l[k] == [False,False,False]:
            val = 0
        elif l[k] == [True,False,False] or l[k] == [False,False,True]:
            val = 1
        elif l[k] == [True,True,False] or l[k] == [False,True,True] or l[k] == [False,True,False]:
            val = 2
        elif l[k] == [True,True,True]:
            val = 3
        p.append(val)

    if 0 in p or 1 in p:
        return [False, p]
    else:
        return [True, p]    

In [3]:
import ipywidgets as w

proc = w.ToggleButtons(
    options=['CTC/IORT', 'Excessive blood loss'],
    description='Procedure:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)


if proc.value == 'CTC/IORT':
    opt = ['pre', 'intra', 'post', 'IC', 'IORT']

elif proc.value == 'Excessive blood loss':
    opt = ['HD unstable', 'HD stable']

phase = w.ToggleButtons(
     options=opt,
     description='Type:',
     disabled=False,
     button_style='', # 'success', 'info', 'warning', 'danger' or ''
)   

In [4]:
procedure = w.Text
def update_prod(change):
    if proc.value == 'CTC/IORT':
        opt = ['pre', 'intra', 'post', 'IC', 'IORT']

    elif proc.value == 'Excessive blood loss':
        opt = ['HD unstable', 'HD stable']
    phase.options = opt 
    

def update_procedure(proc, phase):    
    procedure.value = [proc,phase] 
    

def update_parameters(Thrombocytes,MEA_ADP,MEA_TRAP,APTT,APTT_Heparinase,PT,PT_Heparinase,Fibrinogen,Rotem_A10_Extem,Rotem_A10_Fibtem,Rotem_A10_Heptem,Rotem_CT_Intem,Rotem_CT_Extem,Rotem_CT_Heptem,CThep_CTin):
    return Thrombocytes,MEA_ADP,MEA_TRAP,APTT,APTT_Heparinase,PT,PT_Heparinase,Fibrinogen,Rotem_A10_Extem,Rotem_A10_Fibtem,Rotem_A10_Heptem,Rotem_CT_Intem,Rotem_CT_Extem,Rotem_CT_Heptem,CThep_CTin


proc.observe(update_prod,'value')

In [5]:
val = ['MEA_ADP','MEA_TRAP','APTT','APTT_Heparinase','PT','PT_Heparinase','Fibrinogen','Rotem_A10_Extem','Rotem_A10_Fibtem','Rotem_A10_Heptem','Rotem_CT_Intem','Rotem_CT_Extem','Rotem_CT_Heptem','CThep/CTin', 'Thrombocytes']

style = {'description_width': 'initial'}

Thrombocytes = w.FloatText(
    value=float('nan'),
    description='$Thrombocytes (nl)$',
    style=style,
    disabled=False
)

MEA_ADP = w.FloatText(
    value=float('nan'),
    description='$MEA_{ADP} (U)$',
    style=style,
    disabled=False
)

MEA_TRAP = w.FloatText(
    value=float('nan'),
    description='$MEA_{TRAP} (U)$',
    style=style,
    disabled=False
)

APTT = w.FloatText(
    value=float('nan'),
    description='$APTT (s)$',
    style=style,
    disabled=False
)


APTT_Heparinase= w.FloatText(
    value=float('nan'),
    description='$APTT_{Heparinase} (s)$',
    style=style,
    disabled=False
)

PT = w.FloatText(
    value=float('nan'),
    description='$PT (s)$',
    style=style,
    disabled=False
)

PT_Heparinase= w.FloatText(
    value=float('nan'),
    description='$PT_{Heparinase} (s)$',
    style=style,
    disabled=False
)

Fibrinogen = w.FloatText(
    value=float('nan'),
    description='$Fibrinogen (g/L)$',
    style=style,
    disabled=False
)

Rotem_A10_Extem= w.FloatText(
    value=float('nan'),
    description='$A10_{Extem} (mm)$',
    style=style,
    disabled=False
)

Rotem_A10_Fibtem= w.FloatText(
    value=float('nan'),
    description='$A10_{Fibtem} (mm)$',
    style=style,
    disabled=False
)

Rotem_A10_Heptem= w.FloatText(
    value=float('nan'),
    description='$A10_{Heptem} (mm)$',
    style=style,
    disabled=False
)


Rotem_CT_Intem= w.FloatText(
    value=float('nan'),
    description='$CT_{Intem} (s)$',
    style=style,
    disabled=False
)

Rotem_CT_Extem= w.FloatText(
    value=float('nan'),
    description='$CT_{Extem} (s)$',
    style=style,
    disabled=False
)

Rotem_CT_Heptem= w.FloatText(
    value=float('nan'),
    description='$CT_{Heptem} (s)$',
    style=style,
    disabled=False
)

CThep_CTin= w.FloatText(
    value=float('nan'),
    description='$CT_{Heptem}/CT_{Intem}$',
    style=style,
    disabled=False
)



#display(Thrombocytes,MEA_ADP,MEA_TRAP,APTT,APTT_Heparinase,PT,PT_Heparinase,Fibrinogen,Rotem_A10_Extem,Rotem_A10_Fibtem,Rotem_A10_Heptem,Rotem_CT_Intem,Rotem_CT_Extem,Rotem_CT_Heptem,CThep_CTin)

In [6]:
import pandas as pd


def row_style(row,sev):
    # 0) none exceeded -> green
    # 1) LB exceeded, co/UB not -> requires attention -> orange
    # 2) LB & co exceeded, UB not -> pathologic state but within error ranges -> red
    # 3) LB, co & UB, definitely pathologic state -> dred
    
    if len(sev[row[0]]) == 2:
        ind = min(sev[row[0]])
    else:
        ind = sev[row[0]][0]
    if ind == 0:
        return pd.Series('background-color: lightgreen', row.index)
    elif ind == 1:
        return pd.Series('background-color: orange', row.index)
    elif ind == 2:
        return pd.Series('background-color: red', row.index)
    elif ind == 3:
        return pd.Series('background-color: darkred', row.index)
    else:
        return pd.Series('',row.index)
        

def clicked(b):
    state = ['Plasma defects:', 'Low fibrinogen:', 'Patient is heparinized:','Thrombocyte disfunction:']
    parameters=p.kwargs
    parameters['CThep/CTin'] = parameters['CThep_CTin']
    del parameters['CThep_CTin']
    curr_state, sev, pathologies = knowledge_base(procedure=procedure.value,parameters=parameters)
    tables = []
    for i in range(len(state)):
        keys = [] #parameter names
        cut_off = [] #cut-off values, taken from pathologies
        values = [] #patient values, taken from 'parameters'
        for j in curr_state[i]:
            keys.append(j)
            if len(pathologies[i][j]) == 3:
                cut_off.append(pathologies[i][j][1] + ' ' + str(pathologies[i][j][0]))
                values.append(parameters[j])
            elif len(pathologies[i][j]) == 6:
                cut_off.append(pathologies[i][j][1] + ' ' + str(pathologies[i][j][0]) + ' & ' + pathologies[i][j][4] + ' ' + str(pathologies[i][j][3]))
                values.append([parameters[j[0]],parameters[j[1]]])
        #order the data for the DataFrame 
        data = []
        for k in range(len(keys)):
            array = [keys[k], values[k], cut_off[k]]
            data.append(array)
        df = pd.DataFrame(data, columns = ['Parameter','Value','Cut-off value'])
        tables.append(df)
    if procedure.value == ['CTC/IORT', 'intra']:
        for i in [1,3]:
            print(state[i])
            display(tables[i].style.apply(row_style, sev=sev[i],axis=1))
            print('\n')            
    else:
        for i in range(len(tables)):
            print(state[i])
            display(tables[i].style.apply(row_style, sev=sev[i],axis=1))
            print('\n')
    

button = w.Button(description="Run algorithm")
acc_1 = w.VBox([proc,phase])
l_acc_2 = w.VBox([Fibrinogen,Rotem_A10_Extem,Rotem_A10_Fibtem,Rotem_A10_Heptem,Rotem_CT_Intem,Rotem_CT_Extem,Rotem_CT_Heptem,CThep_CTin])
r_acc_2 = w.VBox([Thrombocytes,MEA_ADP,MEA_TRAP,APTT,APTT_Heparinase,PT,PT_Heparinase])
acc_2 = w.HBox([l_acc_2,r_acc_2])

p = w.interactive(update_parameters, MEA_ADP = MEA_ADP,MEA_TRAP = MEA_TRAP,PT = PT,PT_Heparinase = PT_Heparinase,APTT = APTT,APTT_Heparinase = APTT_Heparinase,Fibrinogen = Fibrinogen,Rotem_A10_Extem = Rotem_A10_Extem,Rotem_A10_Fibtem = Rotem_A10_Fibtem,Rotem_A10_Heptem = Rotem_A10_Heptem,Rotem_CT_Extem = Rotem_CT_Extem,Rotem_CT_Heptem = Rotem_CT_Heptem,Rotem_CT_Intem = Rotem_CT_Intem,CThep_CTin = CThep_CTin,Thrombocytes = Thrombocytes)
s = w.interactive(update_procedure, proc=proc,phase=phase)
button.on_click(clicked)

accordion = w.Accordion(children = [acc_1,acc_2])
accordion.set_title(0,'Procedure')
accordion.set_title(1,'Parameters')

CDSS = w.VBox([accordion,button])

***
# Clinical decision support system for the administration of hemostatics
## 1. Using the CDSS

- Select the relevant procedure (divided between *CTC/IORT* and *Excessive blood loss*) and the subprocedure.
- Fill in the results of the perioperative coagulation tests as the patient's parameters.
    - It is possible to leave a cell empty. The corresponding parameter will then not be included in the evaluation.
- Press **Run Algorithm** to start the CDSS.
    - If one would like to run the CDSS a second time with the same values, please reinsert a single value within the **Parameters** section, as this will cause errors otherwise.
    - To clear the output, press the *Restart kernel* button again. 

## 2. Interpreting the results

- The parameters are divided into four categories or *coagulopathies*:
    - *Plasma defects*
    - *Low fibrinogen*
    - *Patient is heparinized*
    - *Thrombocyte dysfunction*
- Each category will then show the relevant parameters in a table according to the procedure that was chosen before. 


- Parameters that are highlighted:
    - <font color=green> **green** </font> has a value that is well outside of its cut-off value and its error bounds.
    - <font color=orange> **orange** </font> has a value that is dangerously nearing its cut-off value, such that the error of the value overlaps with the cut-off value itself.
    - <font color=red> **red** </font> has a value that has exceeded its cut-off value, however, it is still within the error of a value outside of the cut-off value.
    - <font color=darkred> **dark red** </font> has a value that has well passed its cut-off value. 
- The error for all methods is assumed to be 10%, except for the MEA, which has an error of 20%.
- Alternatively, the color coding can also be interpreted as a measure of the severity of which the patient's value differs from a normal state.
- Parameters that have been highlighted as <font color=red> **red** </font> and <font color=darkred> **dark red** </font> indicate that the patient suffers from the respective coagulopathy.

- Based on these results and the guidelines of the Catharina hospital, the CDSS will also give an advice under the header *Advice*. Multiple advices are possible, while it is also possible it will display nothing when no pathological state has been found.  

***

In [7]:
display(CDSS)


VkJveChjaGlsZHJlbj0oQWNjb3JkaW9uKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShUb2dnbGVCdXR0b25zKGRlc2NyaXB0aW9uPXUnUHJvY2VkdXJlOicsIG9wdGlvbnM9KCdDVEMvSU9SVCfigKY=
